In [2]:
# Import necessary libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from api_credentials import client_id, client_secret
import pandas as pd
from functions import get_playlist_tracks_and_artists, find_tracks_positions_in_playlists
import json
import re

# Load list of playlists from JSON file
with open('playlists.json', 'r') as file:
    playlists_dict = json.load(file)

# Initialise the Spotify client with client credentials for public data access
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

import logging
# Configure logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

### Fetch New Music Friday AU & NZ tracks, and then find which addtional playlists they are in (and their positions):

In [3]:
# New Music Friday AU & NZ playlist 
playlist_id = '37i9dQZF1DWT2SPAYawYcO'

# Fetches track names and artist names from New Music Friday AU & NZ
# Returns a list of tuples, each containing a track name and concatenated artist names.
# Example, [('Foam', 'Royel Otis'),('One More Night', 'KUČKA, Flume')]
track_details = get_playlist_tracks_and_artists(sp, playlist_id)

# Uses`track_details` from above and `playlists_dict' (loaded JSON file)
# Finds the positions of each track in multiple playlists

track_positions = find_tracks_positions_in_playlists(sp, track_details, playlists_dict)

Checking playlist: Top 50 Australia
Checking playlist: New Music Friday AU & NZ
Checking playlist: Hot Hits Australia
Checking playlist: Front Left
Checking playlist: A1
Checking playlist: Dance Generation
Checking playlist: Get Popped!
Checking playlist: R&B Connect
Checking playlist: triple j's New Music Hitlist
Checking playlist: The Flavour
Checking playlist: Fresh Country
Checking playlist: New Dance Beats
Checking playlist: Pop n' Fresh
Checking playlist: R&B Feels
Checking playlist: Beats n' Bars
Checking playlist: Indie Arrivals
Checking playlist: Rock Out.
Checking playlist: Mellow Styles
Checking playlist: POP EDGE
Checking playlist: The Drip
Checking playlist: come thru
Checking playlist: Alt Here
Checking playlist: Breaking Hits
Checking playlist: Chilled Hits
Checking playlist: Vibe Check
Checking playlist: crush
Checking playlist: the hybrid
Checking playlist: Just Chill
Checking playlist: Morning Motivation
Checking playlist: Lazy Sunday
Checking playlist: Today's Top Hi

`track_positions` example:

```json
{
  "TEXAS HOLD 'EM - Beyoncé": {
    "track_name": "TEXAS HOLD 'EM",
    "artist_name": "Beyoncé",
    "playlists": [
      {"playlist": "Top 50 Australia", "position": 5},
      {"playlist": "New Music Friday AU & NZ", "position": 1},
      {"playlist": "Hot Hits Australia", "position": 10}
    ]
  }
}


### Fetch Follower Count For Each Playlist:

In [7]:
# Fetching playlist follower counts
# Dictionary to store follower counts
playlist_followers = {}

# Fetching follower counts
for playlist_name, playlist_id in playlists_dict.items():
    playlist = sp.playlist(playlist_id)
    follower_count = playlist['followers']['total']
    playlist_followers[playlist_name] = follower_count
    if 'followers' not in playlist:
        logging.warning(f"Followers data missing in the playlist data for {playlist_id}")
print("Playlist followers data have been fetched")
# # Printing the result

# for playlist_name, followers in playlist_followers.items():
#     print(f"{playlist_name}: {followers:,} followers")

Playlist followers data have been fetched


### Create and save Dataframe with Spotify data:

In [4]:

rows = []

for track_id, track_info in track_positions.items():
    artist_name = track_info['artist_name']
    track_name = track_info['track_name']
    for playlist_info in track_info['playlists']:
        playlist_name = playlist_info['playlist']
        position = playlist_info['position']
        # Fetch the actual follower count using the playlist name
        followers = playlist_followers.get(playlist_name, 0)  # Default to 0 if playlist not found
        rows.append({
            'Artist': artist_name,
            'Title': track_name,
            'Playlist': playlist_name,
            'Position': position,
            'Followers': followers
        })

# Convert the list of rows into a DataFrame
df = pd.DataFrame(rows)

df.to_csv('streamlit.csv', index=False)

### Fetch Playlist Images:

In [ ]:
cover_art_dict = {}

for playlist_name, playlist_id in playlists_dict.items():

    playlist_data = sp.playlist(playlist_id)

    # Fetching playlist cover image URL
    cover_image_url = playlist_data['images'][0]['url'] if playlist_data['images'] else 'No image available'

    # append to dictionary 
    cover_art_dict[playlist_name] = cover_image_url


### Fetch Cover Artist Details:

In [27]:
# Initialize the dictionary outside of the loop
cover_artist_dict = {}

for playlist_name, playlist_id in playlists_dict.items():
    # Fetch playlist data from Spotify
    playlist = sp.playlist(playlist_id)

    # Extract the required information
    playlist_description = playlist.get('description', 'No description available')

    # Use regex for case-insensitive search for 'Cover: ' and extract the cover artist name
    match = re.search(r'cover:\s*(.*?)$', playlist_description, re.IGNORECASE)
    if match:
        cover_artist = match.group(1)  # Extract the matched artist name
        # Optional: if you want to stop at the first period (.)
        if '.' in cover_artist:
            cover_artist = cover_artist.split('.')[0]
        # Add to the dictionary only if the cover artist is meaningful (not 'No cover artist found')
        if cover_artist.strip().lower() != "no cover artist found":
            cover_artist_dict[playlist_name] = cover_artist

### Remove Image URLs from `cover_art_dict` that don't have a Cover Artist. As they will not be needed. Only Cover Art featuring an artist is useful. 

In [30]:
# Create a new dictionary that will only include matching keys
filtered_cover_art_dict = {}

# Loop through the cover_art_dict
for playlist_name in cover_art_dict:
    # Check if the current key also exists in cover_artist_dict
    if playlist_name in cover_artist_dict:
        # Add it to the new dictionary
        filtered_cover_art_dict[playlist_name] = cover_art_dict[playlist_name]

### Saving dictionary's to JSON files for later import use into Streamlit.

In [31]:
data = {
    'filtered_cover_art_dict': filtered_cover_art_dict,
    'cover_artist_dict': cover_artist_dict
}

# Write the combined dictionary to a file
with open('cover_art_data.json', 'w') as f:
    json.dump(data, f, indent=4)